### *STEP1:* 将原来6G的xml文件切分成平均1M的小文档

In [3]:
RUN = False
if(RUN):  
    fin = open('zhwiki-20171020-pages-articles-multistream.xml',encoding='utf8')
    for i in range(10000000):      
        readin = fin.readlines(100*1000)
        if(not readin):
            break
        fout = open('parts/part_'+str(i),'w',encoding = 'utf8')
        fout.writelines(readin)
        # fout.writelines(re.sub("[\[\]\'\"\@#$%^&*()<>：{}|/_.+-—，,。!！ ？、~@#￥%……&*（）]+", "",s))
        fout.close()
    fin.close()
    partsNum = i
    print(partsNum)

49125


**FUNCTION** 判断一个unicode是否为汉字

In [3]:
def is_ch(uchar):
        """判断一个unicode是否是汉字"""
        if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
                return True
        else:
                return False

### *STEP2:* 导入GBK编码表，构造一个dic，将汉字映射到整数

In [97]:
with open('GBK',encoding='utf8') as fin:
    chList = []
    for line in fin.readlines():
        for word in line:
            if(is_ch(word)):
                chList.append(word)

                
chNum = len(chList)
print('词典大小：',chNum)
ch2int = {}
idx = 0
with open('ch2int','w',encoding='utf8') as fout:  
    for w in chList:
        ch2int[w] = idx
        fout.write(w+'\t'+str(idx)+'\n')
        idx += 1


词典大小： 20902


**CLASS** 定义类BitSet 用于集合运算

In [103]:
class BitSet:
    def __init__(self, elementNum):
        self.bytesNum = round(elementNum/8)
        self.set = bytearray(self.bytesNum)
        
    def insert(self, index):
        if(index >= 0 and index>>3 < self.bytesNum):
            self.set[index >> 3] = self.set[index >> 3] | (1 << int(index & 7)) 
            return True;
        return False;
    
    def isElement(self, index):
        if(index >= 0 and index>>3 < self.bytesNum and (self.set[index >> 3] & (1<<(index & 7)))):
            return True
        return False
        
    def AND(self, bs):
        a = BitSet(self.bytesNum*8)
        for i in range(self.bytesNum):
            a.set[i] = self.set[i] & bs.set[i]
        return a

### *STEP3:* 给每一个汉字创建倒排索引，对应包含该汉字的文档

In [104]:
partsNum = 49125
bytesNum = round(partsNum/8)

invertedIndex = [BitSet(partsNum) for i in range(chNum)]
print('每个字的倒排索引表的字节数：',invertedIndex[0].bytesNum)

每个字的倒排索引表的字节数： 6141


**FUNCTION** 显示进度条

In [105]:
import sys,time

def processBar(index, totalNum):
    bar = '#'*int((index/totalNum)*50)
    sys.stdout.write(str(int((index/totalNum)*100))+'%  ||'+bar+'->'+"\r")
    #sys.stdout.flush()

遍历所有文档的所有字符，建立倒排索引

In [21]:

#for partIdx in range(partsNum):
for partIdx in range(100):   
    processBar(partIdx, partsNum)
    
    with open('parts/part_'+str(partIdx),encoding='utf8') as f:
        for word in f.read():
            try:
                if(is_ch(word)):
                    invertedIndex[ch2int[word]].insert(partIdx)
            except KeyError as err:
                print('KEY ERROR'+str(err))

for chIdx in range(chNum):    
    processBar(chIdx, chNum)
    with open('iidx/ch_'+str(chIdx)+'.iidx','w',encoding='utf8') as f:
            for j in invertedIndex[chIdx].set:
                f.write(str(j)+' ')

测试：读取倒排索引

In [106]:
def readIidx(file):
    with open(file,encoding='utf8') as f:
        s = f.read()[:-1].split(' ')
    bSet = BitSet(len(s)*8)
    for i in range(bSet.bytesNum):
        bSet.set[i] = int(s[i])
    return bSet

f = 'iidx/ch_0.iidx'
b = readIidx(f)
print(b.set[:20])

bytearray(b'\x07\x08\x06\x00\x002\n\x0c\x00\x18\x08\x14\x06\x00\x00\x00\x00\x00\x00\x00')


### *STEP4:* 建立每个文档的 tf 表

In [107]:
# tf 
RUN_tf = False

if(RUN_tf):
    for partIdx in range(partsNum):

        tf = [0.0 for i in range(chNum)]
        total = 0

        with open('parts/part_'+str(partIdx),encoding='utf8') as f:
            for word in f.read():
                if(is_ch(word)):
                    tf[ch2int[word]] += 1
                    total += 1

        total = max(total, 1)
        for i in range(chNum):
            tf[i] /= total

        with open('parts/tf/part_'+str(partIdx)+'.tf','w',encoding='utf8') as fout:
            for i in tf:
                fout.write(str(i)+' ')
                
        processBar(partIdx, partsNum)


### *STEP5:* 计算每个单词的 idf 值

In [35]:
# idf(ch) = log(total DocumentNum / Num of documents containing ch)
import math

idf = [0.0 for i in range(chNum)]

for chIdx in range(chNum):
    processBar(chIdx, chNum)
    num = 0
    iidxFile = 'iidx/ch_'+str(chIdx)+'.iidx'
    bs = readIidx(iidxFile)
    for i in range(partsNum):
        if(bs.isElement(i)):
            num += 1
    idf[chIdx] = math.log(partsNum/(num+1)) # divide by zero

with open('iidx/idf','w',encoding='utf8') as fout:
    for i in idf:
        fout.write(str(i)+' ')
    

### *STEP6:* 搜索到包含关键词的文档集, 将搜索到的文档集根据 tf-idf 值排序

In [110]:
def mask(bytes):
    r = BitSet(bytes*8)
    for i in range(bytes):
        r.set[i] = 255
    return r

bsMask = mask(bytesNum) # 掩码

def getIdf(partIdx):
    file = 'iidx/idf'
    with open(file, encoding='utf8') as f:
        s = f.read().split()
    return float(s[partIdx])

def tfidf(partIdx, keyList):
    file = 'parts/tf/part_'+str(partIdx)+'.tf'
    tf = 0
    with open(file,encoding='utf8') as f:
        s = f.read().split()
        for i in keyList:
            tf += float(s[i])
    idf = getIdf(partIdx)
    return tf*idf

def search(query):
    keyList = []
    targets = bsMask
    for word in query:
        if(is_ch(word)):
            keyList.append(ch2int[word])
            targets = targets.AND(readIidx('iidx/ch_'+str(ch2int[word])+'.iidx'))
            
    result = []
    for i in range(partsNum):
        if(targets.isElement(i)):
            result.append(i)
    
    # rearrange
    tuples = []
    for doc in result:
        tuples.append((doc, tfidf(doc,keyList)))
    
    result = sorted(tuples, key = lambda x:x[1], reverse=True)
        
    return result

r = search('蒋介石毛泽东周恩来邓小平中国台湾国名党共产党')
print("（文档号，TFIDF值）")
for i in r[:10]:
    print(i)

（文档号，TFIDF值）
(95, 0.7542362578147894)
(32, 0.6203344862114203)
(37, 0.5982827394017244)
(39, 0.5551600755994012)
(30, 0.5514309729312883)
(44, 0.4623889179372404)
(33, 0.41754225957672414)
(94, 0.40984699051540074)
(78, 0.3926799381317715)
(64, 0.29895649752851083)


假如每次只取出前十名，每篇文档大小100kb，总共只占用1M内存，完全可以将所有文字体取出来分析，找到最佳匹配的句子